<a href="https://colab.research.google.com/github/Dinmir331/Semester8_LB3/blob/main/S8_CV_LB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

3 лаба -- Дано видео, нужно его считать (на видео бегает мыш по полу), отделить мышь от пола и найти её центр на каждом кадре видеоряда используя код разумеется, а не покадровое разбиение с указанием где центр вручную.


1. **Вычитание фона** — выделение мыши через медианное усреднение кадров и морфологию.  
2. **Трекинг** — отслеживание объекта методом корреляции шаблонов с адаптацией.  
3. **Обработка кадров** — бинаризация, морфология, фильтрация контуров по площади/пропорциям.  
4. **Визуализация** — отрисовка bounding box, центра и траектории в GIF.  
5. **Пакетная обработка** — автоматическая обработка 10 видео с выводом статистики.

# Импорт необходимых библиотек

In [84]:
import numpy as np                  # Для работы с массивами и математическими операциями
import cv2                          # Основная библиотека для компьютерного зрения
import imageio                      # Для создания GIF-анимаций
from google.colab import drive      # Для подключения Google Drive

# Монтируем Google Drive для доступа к файлам
drive.mount('/content/drive', force_remount=True)  # Подключение облачного хранилища Google Drive

Mounted at /content/drive


# Инициализия видеопотка и возврат параметров видео

In [85]:
def initialize_video(video_path):
    # Открываем видеофайл с помощью OpenCV
    cap = cv2.VideoCapture(video_path)  # Создаем объект захвата видео
    if not cap.isOpened():  # Проверка, удалось ли открыть видео
        raise ValueError(f"Ошибка открытия видео: {video_path}")  # Выбрасываем ошибку, если видео не открыто

    # Определяем исходное разрешение видео (ширина и высота)
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Получаем ширину кадра
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Получаем высоту кадра

    return cap, original_width, original_height  # Возвращаем объект захвата и размеры кадра

# Обучение вычитателя фона на начальных кадрах

In [86]:
def calculate_static_background(cap, initial_frames, learning_rate, median_blur, morph_iterations):
    ret, frame = cap.read()  # Читаем первый кадр видео

    # Сбор кадров для медианного усреднения
    buffer = []  # Создаем список для хранения обработанных кадров
    for _ in range(initial_frames):  # Проходим по первым initial_frames кадрам
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Преобразуем кадр в градации серого
        gray = cv2.medianBlur(gray, median_blur)  # Применяем медианное размытие для уменьшения шума
        buffer.append(gray)  # Добавляем обработанный кадр в буфер
        ret, frame = cap.read()  # Читаем следующий кадр
        if not ret:  # Если кадры закончились, прерываем цикл
            break

    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Сбрасываем позицию чтения кадров к началу видео

    # Вычисление медианного фона
    background = np.median(buffer, axis=0).astype(np.uint8)  # Вычисляем медиану всех кадров в буфере

    # Улучшение фона с помощью морфологических операций
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # Создаем эллиптическое ядро для морфологии
    background = cv2.morphologyEx(background, cv2.MORPH_CLOSE, kernel, iterations=morph_iterations)  # Закрываем мелкие дыры

    return background  # Возвращаем модель фона

# Создание трекера

In [87]:
def csrt_tracking(frame, roi, tracker_state=None):
    """
    Реализация CSRT-подобного трекинга через корреляцию шаблонов
    :param frame: Текущий кадр
    :param roi: ROI-область в формате (x, y, w, h)
    :param tracker_state: Состояние трекера (для сохранения шаблона)
    :return: Новый bbox, состояние трекера, статус успеха
    """

    if tracker_state is None:  # Если трекер еще не инициализирован
        # Инициализация трекера
        x, y, w, h = roi  # Распаковываем координаты и размеры ROI
        template = frame[y:y+h, x:x+w]  # Вырезаем область интереса (ROI) из кадра
        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)  # Преобразуем ROI в градации серого
        template_gray = cv2.GaussianBlur(template_gray, (7, 7), 0)  # Применяем Гауссово размытие для уменьшения шума
        return {'template': template_gray}, (x, y, w, h), True  # Возвращаем шаблон и координаты ROI

    # Обновление трекера
    template = tracker_state['template']  # Получаем текущий шаблон из состояния трекера
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Преобразуем текущий кадр в градации серого
    frame_gray = cv2.GaussianBlur(frame_gray, (7, 7), 0)  # Применяем Гауссово размытие для уменьшения шума

    # Поиск совпадений методом корреляции
    result = cv2.matchTemplate(frame_gray, template, cv2.TM_CCOEFF_NORMED)  # Ищем совпадения шаблона в кадре
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)  # Находим максимальное значение корреляции

    if max_val < 0.5:  # Если корреляция ниже порога, считаем, что объект потерян
        return None, None, False  # Возвращаем статус неудачи

    # Вычисление новых координат объекта
    h_template, w_template = template.shape[:2]  # Получаем размеры шаблона
    x, y = max_loc  # Координаты максимального совпадения
    new_bbox = (x, y, w_template, h_template)  # Формируем новый bounding box

    # Обновление шаблона каждые 5 кадров
    if np.random.randint(5) == 0:  # Случайно выбираем момент для обновления шаблона
        new_template = frame[y:y+h_template, x:x+w_template]  # Вырезаем новую область интереса
        new_template_gray = cv2.cvtColor(new_template, cv2.COLOR_BGR2GRAY)  # Преобразуем в градации серого
        new_template_gray = cv2.GaussianBlur(new_template_gray, (7, 7), 0)  # Применяем Гауссово размытие
        tracker_state['template'] = new_template_gray  # Обновляем шаблон в состоянии трекера

    return tracker_state, new_bbox, True  # Возвращаем обновленное состояние, новый bbox и статус успеха

# Обработка кадров для поиска объекта

In [88]:
def process_frame(frame, background, var_threshold, min_area, max_aspect_ratio):
    # Преобразование текущего кадра в градации серого для упрощения обработки
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Преобразуем BGR-изображение в градации серого

    # Вычисление маски движения путем вычитания фона
    fgmask = cv2.absdiff(gray_frame, background)  # Находим абсолютную разницу между текущим кадром и фоном
    _, fgmask = cv2.threshold(fgmask, var_threshold, 255, cv2.THRESH_BINARY)  # Бинаризация разницы (выделяем движущиеся области)

    # Применение Гауссова размытия для уменьшения шума на изображении
    gray = cv2.GaussianBlur(gray_frame, (7, 7), 0)  # Размытие с ядром 7x7 для сглаживания
    _, bw_mask = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY_INV)  # Бинаризация по порогу яркости (инвертированная маска)

    # Комбинирование масок для улучшения выделения объекта
    combined_mask = cv2.bitwise_and(fgmask, bw_mask)  # Логическое "И" между масками движения и яркости

    # Морфологические операции для улучшения качества маски
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))  # Создаем эллиптическое ядро размером 9x9
    gray = cv2.erode(gray, kernel, iterations=2)  # Эрозия для удаления мелких шумов и затемнения
    gray = cv2.dilate(gray, kernel, iterations=2)  # Дилатация для соединения частей объекта и осветления

    # Дополнительная обработка комбинированной маски
    combined_mask = cv2.bitwise_and(fgmask, bw_mask)  # Повторное комбинирование масок
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel, iterations=3)  # Операция "открытия" для устранения шума
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel, iterations=3)  # Операция "закрытия" для заполнения дыр

    # Поиск контуров на основе улучшенной маски
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # Поиск внешних контуров
    valid_contours = []  # Список для хранения валидных контуров
    for cnt in contours:
        area = cv2.contourArea(cnt)  # Вычисляем площадь контура
        if area < min_area:  # Фильтруем контуры по минимальной площади (игнорируем маленькие)
            continue

        # Получаем координаты ограничивающего прямоугольника
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / h  # Вычисляем соотношение сторон

        # Фильтрация по соотношению сторон (исключаем слишком продолговатые или узкие объекты)
        if aspect_ratio > max_aspect_ratio or aspect_ratio < 1 / max_aspect_ratio:
            continue

        valid_contours.append(cnt)  # Добавляем валидный контур в список

    mask = combined_mask.astype(np.uint8)  # Преобразуем маску в формат uint8 для дальнейшего использования

    return mask, valid_contours  # Возвращаем маску и список валидных контуров

# Отрисовка траектории на кадре

In [89]:
def draw_trajectory(frame, trajectory, color=(255, 0, 255), thickness=2):
    if len(trajectory) > 1:  # Проверяем, есть ли хотя бы две точки для отрисовки линии
        pts = np.array(trajectory, np.int32).reshape((-1, 1, 2))  # Преобразуем список точек в массив NumPy
        cv2.polylines(frame, [pts], False, color, thickness)  # Рисуем ломаную линию траектории на кадре
        # Параметры: frame — исходный кадр, [pts] — массив точек, False — замкнутая линия, color — цвет, thickness — толщина

# Основная функция трекинга мыши


In [90]:
def track_mouse(video_path, output_gif, mask_gif_path,
               var_threshold=10, # Порог чувствительности вычитания фона (меньше = чувствительнее)
               detection_interval=8,  # Кадров между полными циклами обнаружения
               min_area=10000, # Минимальная площадь объекта для обнаружения (фильтрация шума)
               max_aspect_ratio=4.0,  # Максимальное соотношение сторон (фильтрация продолговатых объектов)
               scale_factor=0.5, # Коэффициент уменьшения (0.5 = 50% от исходного размера изображения)
               frame_rate=30.0,  # Желаемая частота кадров
               frame_skip=2 # Частота кадров принимаемых в GIF-анимацию (чем больше — тем быстрее видео)
              ):
    # Инициализация видеопотока и получение параметров видео
    cap, original_width, original_height = initialize_video(video_path)  # Открываем видеофайл
    # Получаем реальную частоту кадров исходного видео
    real_fps = cap.get(cv2.CAP_PROP_FPS)  # Извлекаем FPS из метаданных видео
    if real_fps <= 0:  # Если FPS не удалось получить, используем значение по умолчанию
        real_fps = frame_rate

    # Расчёт длительности одного кадра в секундах
    duration = (1.0 / real_fps)  # Длительность кадра для GIF

    # Настройка модели фона
    background = calculate_static_background(
      cap,
      initial_frames=30,  # Используем первые 30 кадров для создания модели фона
      learning_rate=0.1,  # Скорость адаптации модели фона
      median_blur=7,      # Усиленное медианное размытие для подавления шума
      morph_iterations=3  # Итерации морфологических операций для улучшения фона
      )

    # Переменные состояния
    tracker_state = None  # Переменная для хранения состояния трекера
    trajectory = []       # Список для хранения координат траектории
    frames = []           # Список для хранения обработанных кадров
    mask_frames = []      # Список для хранения масок объекта

    # Основной цикл обработки видео
    while True:
        ret, frame = cap.read()  # Чтение следующего кадра
        if not ret:  # Если кадры закончились, завершаем цикл
            break

        # Обработка кадра для выделения движущегося объекта
        mask, valid_contours = process_frame(
            frame, background, var_threshold, min_area, max_aspect_ratio)  # Вычисление маски и контуров
        mask_frames.append(cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR))  # Преобразование маски в RGB для сохранения

        # Обработка трекера (если он активен)
        if tracker_state is not None:
            tracker_state, bbox, success = csrt_tracking(frame, bbox, tracker_state)  # Обновление трекера
            if success:  # Если объект успешно найден
                x, y, w, h = map(int, bbox)  # Получаем координаты bounding box
                cx = x + w // 2             # Центр объекта по оси X
                cy = y + h // 2             # Центр объекта по оси Y

                SMOOTHING_FACTOR = 0.5      # Коэффициент сглаживания траектории
                MAX_SMOOTH_POINTS = 3       # Количество точек для усреднения

                # Плавное обновление координат центра объекта
                if len(trajectory) >= MAX_SMOOTH_POINTS:
                    smooth_cx = int(np.mean([p[0] for p in trajectory[-MAX_SMOOTH_POINTS:]]) * (1 - SMOOTHING_FACTOR) + cx * SMOOTHING_FACTOR)
                    smooth_cy = int(np.mean([p[1] for p in trajectory[-MAX_SMOOTH_POINTS:]]) * (1 - SMOOTHING_FACTOR) + cy * SMOOTHING_FACTOR)
                    cx, cy = smooth_cx, smooth_cy  # Обновляем координаты с учётом сглаживания

                trajectory.append((cx, cy))  # Добавляем новые координаты в траекторию

                # Отрисовка трекера на кадре
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Рисуем прямоугольник вокруг объекта
                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)  # Рисуем центр объекта
                cv2.putText(frame, "Tracking", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)  # Добавляем текст "Tracking"
            else:
                tracker_state = None  # Сброс трекера при потере объекта

        # Периодическое обнаружение объекта
        if not tracker_state or len(trajectory) % detection_interval == 0:
            if valid_contours:  # Если есть валидные контуры
                main_contour = max(valid_contours, key=cv2.contourArea)  # Выбираем контур с максимальной площадью
                x, y, w, h = cv2.boundingRect(main_contour)  # Получаем координаты bounding box
                tracker_state, bbox, _ = csrt_tracking(frame, (x, y, w, h))  # Инициализация трекера

        draw_trajectory(frame, trajectory)  # Отрисовка траектории на кадре
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # Сохраняем обработанный кадр в RGB формате

    # Пропуск кадров для оптимизации GIF
    frames = frames[::frame_skip]  # Пропускаем кадры согласно frame_skip
    mask_frames = mask_frames[::frame_skip]  # Пропускаем кадры масок

    # Сохранение результатов в GIF
    size = (int(original_width * scale_factor), int(original_height * scale_factor))  # Новый размер кадра
    with imageio.get_writer(output_gif, duration=duration, loop=0, subrectangles=True) as writer:
        for frame in frames:  # Запись обработанных кадров в GIF
            writer.append_data(cv2.resize(frame, size))  # Масштабирование кадра перед записью

    with imageio.get_writer(mask_gif_path, duration=duration, loop=0, subrectangles=True) as writer:
        for mask in mask_frames:  # Запись масок в GIF
            writer.append_data(cv2.resize(mask, size))  # Масштабирование маски перед записью

    # Вывод статистики
    print(f"Скорость видео: {real_fps:.1f} FPS, масштаб: {scale_factor}")
    print(f"Кадров в видео: {len(frames)}, масок: {len(mask_frames)}")

    cap.release()  # Освобождение ресурсов (закрытие видеофайла)
    print(f"Готово! Сохранено: {output_gif} и {mask_gif_path}")

# Вызов функции

In [92]:
# Базовые пути для обработки видео
base_video_path = '/content/drive/MyDrive/Colab Notebooks/LB3_Mouse{}.mp4'  # Шаблон пути к исходным видео
base_output_gif = '/content/drive/MyDrive/Colab Notebooks/LB3_Mouse_tracking{}.gif'  # Шаблон пути для сохранения GIF
base_mask_gif = '/content/drive/MyDrive/Colab Notebooks/LB3_Mouse_mask{}.gif'  # Шаблон пути для сохранения масок

# Цикл обработки нескольких видео
for i in range(1, 11):  # Обрабатываем видео с номерами от 1 до 10
    video_path = base_video_path.format(i)  # Формируем путь к текущему видео
    output_gif = base_output_gif.format(i)  # Формируем путь для сохранения GIF
    mask_gif_path = base_mask_gif.format(i)  # Формируем путь для сохранения масок

    print(f"Начинаю обработку видео {i}")  # Выводим сообщение о начале обработки

    try:
        track_mouse(
            video_path=video_path,  # Путь к исходному видео
            output_gif=output_gif,  # Путь для сохранения GIF
            mask_gif_path=mask_gif_path  # Путь для сохранения масок
        )
        print(f"Успешно завершена обработка видео {i}\n")  # Выводим сообщение об успешной обработке
    except Exception as e:
        print(f"Ошибка при обработке видео {i}: {str(e)}\n")  # Выводим сообщение об ошибке

Начинаю обработку видео 1
Скорость видео: 30.0 FPS, масштаб: 0.5
Кадров в видео: 102, масок: 102
Готово! Сохранено: /content/drive/MyDrive/Colab Notebooks/LB3_Mouse_tracking1.gif и /content/drive/MyDrive/Colab Notebooks/LB3_Mouse_mask1.gif
Успешно завершена обработка видео 1

Начинаю обработку видео 2
Скорость видео: 29.9 FPS, масштаб: 0.5
Кадров в видео: 160, масок: 160
Готово! Сохранено: /content/drive/MyDrive/Colab Notebooks/LB3_Mouse_tracking2.gif и /content/drive/MyDrive/Colab Notebooks/LB3_Mouse_mask2.gif
Успешно завершена обработка видео 2

Начинаю обработку видео 3
Скорость видео: 29.9 FPS, масштаб: 0.5
Кадров в видео: 151, масок: 151
Готово! Сохранено: /content/drive/MyDrive/Colab Notebooks/LB3_Mouse_tracking3.gif и /content/drive/MyDrive/Colab Notebooks/LB3_Mouse_mask3.gif
Успешно завершена обработка видео 3

Начинаю обработку видео 4
Скорость видео: 30.0 FPS, масштаб: 0.5
Кадров в видео: 156, масок: 156
Готово! Сохранено: /content/drive/MyDrive/Colab Notebooks/LB3_Mouse_trac

# Основная функция трекинга мыши mp4

In [91]:
# не учитывать

# def track_mouse(video_path, output_mp4, mask_mp4_path,
#                var_threshold=10, # Порог чувствительности вычитания фона (меньше = чувствительнее)
#                detection_interval=8,  # Кадров между полными циклами обнаружения
#                min_area=10000, # Минимальная площадь объекта для обнаружения (фильтрация шума)
#                max_aspect_ratio=4.0,  # Максимальное соотношение сторон (фильтрация продолговатых объектов)
#                scale_factor=1.0, # Коэффициент уменьшения (0.5 = 50% от исходного размера изображения)
#                frame_rate=30.0,  # Желаемая частота кадров
#                speed_factor=1.0  # Множитель скорости (0.5 - ускорить в 2 раза, 2 - замедлить)
#               ):

#     cap, original_width, original_height = initialize_video(video_path) # Открываем видеофайл

#     # Настройка вычитателя фона
#     background = calculate_static_background(
#       cap,
#       initial_frames=30,
#       learning_rate=0.1,
#       median_blur=7,          # Усиленное размытие
#       morph_iterations=3      # Больше морфологии
#       )

#     # Переменные состояния
#     tracker_state = None # Переменная для хранения трекера
#     trajectory = [] # Хранение координат траектории

#     # Получаем реальную частоту кадров исходного видео
#     real_fps = cap.get(cv2.CAP_PROP_FPS)
#     if real_fps <= 0:
#         real_fps = frame_rate  # Значение по умолчанию, если не удалось получить

#     # Сохранение результатов
#     size = (int(original_width*scale_factor), int(original_height*scale_factor))

#     # Настройка записи видео
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')

#     # Расчет целевой частоты кадров
#     target_fps = real_fps * speed_factor

#     # Основное видео
#     out_video = cv2.VideoWriter(
#         output_mp4,
#         fourcc,
#         target_fps,
#         size
#     )

#     # Видео с масками
#     out_mask = cv2.VideoWriter(
#         mask_mp4_path,
#         fourcc,
#         target_fps,
#         size
#     )

#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Обрабатываем маску на каждом кадре
#         mask, valid_contours = process_frame(
#             frame, background, var_threshold, min_area, max_aspect_ratio)

#         # Обработка трекера
#         if tracker_state is not None:
#             tracker_state, bbox, success = csrt_tracking(frame, bbox, tracker_state)
#             if success:
#                 x, y, w, h = map(int, bbox)         # Получаем координаты
#                 cx = x + w//2                       # Центр объекта по X
#                 cy = y + h//2                       # Центр объекта по Y

#                 SMOOTHING_FACTOR = 0.3  # Коэффициент сглаживания (0.0 - максимальное сглаживание, 1.0 - без сглаживания)
#                 MAX_SMOOTH_POINTS = 3   # Количество точек для усреднения

#                 # Плавное обновление координат
#                 if len(trajectory) >= MAX_SMOOTH_POINTS:
#                   # Берем среднее из последних MAX_SMOOTH_POINTS точек
#                   smooth_cx = int(np.mean([p[0] for p in trajectory[-MAX_SMOOTH_POINTS:]]) * (1 - SMOOTHING_FACTOR) + cx * SMOOTHING_FACTOR)
#                   smooth_cy = int(np.mean([p[1] for p in trajectory[-MAX_SMOOTH_POINTS:]]) * (1 - SMOOTHING_FACTOR) + cy * SMOOTHING_FACTOR)
#                   cx, cy = smooth_cx, smooth_cy

#                 trajectory.append((cx, cy))

#                 # Отрисовка трекера
#                 cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2) # Прямоугольник вокруг объекта
#                 cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1) # Центр объекта
#                 cv2.putText(frame, "Tracking", (x, y-10),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
#             else:
#                 tracker_state = None # Сброс трекера при потере

#         # Периодическое обнаружение объекта
#         if not tracker_state or len(trajectory) % detection_interval == 0:
#             if valid_contours:
#                 main_contour = max(valid_contours, key=cv2.contourArea)
#                 x, y, w, h = cv2.boundingRect(main_contour)
#                 tracker_state, bbox, _ = csrt_tracking(frame, (x,y,w,h))  # Инициализация

#         draw_trajectory(frame, trajectory)

#         # Запись кадров вместо сохранения в списки
#         resized_frame = cv2.resize(frame, size)
#         resized_mask = cv2.resize(cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR), size)

#         out_video.write(resized_frame)
#         out_mask.write(resized_mask)

#     # Освобождение ресурсов
#     out_video.release()
#     out_mask.release()
#     cap.release()

#     print(f"Скорость видео: {real_fps:.1f} FPS, масштаб: {scale_factor}")

#     print(f"Готово! Сохранено: {output_mp4} и {mask_mp4_path}")

# Вызов функции mp4

In [93]:
# не учитывать

# # Базовые пути
# base_video_path = '/content/drive/MyDrive/Colab Notebooks/LB3_Mouse{}.mp4' # Путь к исходному видео
# base_output_mp4 = '/content/drive/MyDrive/Colab Notebooks/LB3_Mouse_tracking{}.mp4' # Путь для сохранения mp4
# base_mask_mp4 = '/content/drive/MyDrive/Colab Notebooks/LB3_Mouse_mask{}.mp4' # Путь для сохранения mp4 бинаризированной маски

# # Цикл обработки
# for i in range(1, 11):  # От 1 до 10 включительно
#     video_path = base_video_path.format(i)
#     output_mp4 = base_output_mp4.format(i)
#     mask_mp4_path = base_mask_mp4.format(i)

#     print(f"Начинаю обработку видео {i}")

#     try:
#         track_mouse(
#             video_path=video_path,
#             output_mp4=output_mp4,
#             mask_mp4_path=mask_mp4_path
#         )
#         print(f"Успешно завершена обработка видео {i}\n")
#     except Exception as e:
#         print(f"Ошибка при обработке видео {i}: {str(e)}\n")